In [5]:
import requests
import json
import pandas as pd
import ta
from pandasgui import show
import ipywidgets as widgets

In [3]:
#Daily Timeframe Data Grabber
def daily_df(ticker):

    API_URL = "https://www.alphavantage.co/query"

    data = {
        "function": "TIME_SERIES_DAILY",
        "symbol": ticker,
        "outputsize": "compact",
        "datatype": "json",
        "apikey": "PVGTC9P6IUS7XLQ9",
        }

    response = requests.get(API_URL, params=data)
    
    if response.status_code == 200:
        stock_data = response.json()
        meta_data = stock_data['Meta Data']
        daily_data = stock_data['Time Series (Daily)']
        daily_dataframe_rev = pd.DataFrame.from_dict(daily_data)
        daily_dataframe = daily_dataframe_rev.transpose()     
        daily_dataframe.columns = ['open', 'high', 'low', 'close', 'volume']
        df_entries2float(daily_dataframe)
        return daily_dataframe

In [4]:
#Converting dataframe entries to floats
def df_entries2float(df):
    for i in df.columns:
        column2convert = df[i]
        column_size = column2convert.size
        for k in range(0,column_size):
            column2convert[k] = float(column2convert[k])
    return df

In [5]:
#Reversing dataframe
def rev(df):
    return df.sort_index(ascending = True, axis = 0)

In [6]:
#Running TA with proper formatting
def tech_indicator(indicator, daily_df):
    indicator_df = pd.DataFrame(indicator)
    indicator_df = indicator_df.sort_index(ascending=False, axis=0)
    resulting_df = daily_df.join(indicator_df)
    return resulting_df

In [7]:
#All technical indicators in use
def run_indicators(df):
    high = rev(df['high'])
    low = rev(df['low'])    
    close = rev(df['close'])
    volume = rev(df['volume'])
    
    df = tech_indicator(ta.momentum.wr(high,low,close),df)
    df = tech_indicator(ta.momentum.money_flow_index(high,low,close,volume),df)
    df = tech_indicator(ta.momentum.stoch_signal(high,low,close),df)
    df = tech_indicator(ta.momentum.tsi(close),df)
    df = tech_indicator(ta.trend.macd(close),df)
    df = tech_indicator(ta.trend.trix(close),df)
    df = tech_indicator(ta.trend.aroon_up(close),df)
    df = tech_indicator(ta.trend.aroon_down(close),df)
    df = tech_indicator(ta.volatility.keltner_channel_hband_indicator(high,low,close),df)
    df = tech_indicator(ta.volatility.keltner_channel_lband_indicator(high,low,close),df)
    df = tech_indicator((ta.volatility.bollinger_hband_indicator(close)),df)
    df = tech_indicator((ta.volatility.bollinger_lband_indicator(close)),df)  
    df = tech_indicator(ta.volume.chaikin_money_flow(high,low,close,volume),df)
    
    df['bbi'] = df['bbihband'].subtract(df['bbilband'])
    df['kci'] = df['kci_hband'].subtract(df['kci_lband'])
    del df['bbihband']
    del df['bbilband']
    del df['kci_hband']
    del df['kci_lband']
    df = tech_indicator(ta.momentum.rsi(close),df)
    
    return df

In [8]:
#Mapping values
def map_values(df):
    length = len(df)
    
    df = df.copy()
    
    #Mapping wr values
    wr = df.columns.get_loc('wr')
    for i in range(length):
        if -100 <= df.iloc[i,wr] <= -80:
            df.iloc[i,wr] = 2
        elif -20<= df.iloc[i,wr] <= 0:
            df.iloc[i,wr] = -2
        elif -30 <= df.iloc[i,wr] < -20:
            df.iloc[i,wr] = -1
        elif -80 < df.iloc[i,wr] <= -70:
            df.iloc[i,wr] = 1
        else:
            df.iloc[i,wr] = 0
            
    #Mapping MFI values
    mfi = df.columns.get_loc('mfi_14')
    for i in range(length):
        if 0 <= df.iloc[i,mfi] <= 20:
            df.iloc[i,mfi] = 2
        elif 80 <= df.iloc[i,mfi] <= 100:
            df.iloc[i,mfi] = -2
        elif 20 <= df.iloc[i,mfi] < 30:
            df.iloc[i,mfi] = 1
        elif 70 < df.iloc[i,mfi] <= 80:
            df.iloc[i,mfi] = -1
        else:
            df.iloc[i,mfi] = 0
            
    #Mapping Stoch_d values
    stoch = df.columns.get_loc('stoch_d')
    for i in range(length):
        if 80 <= df.iloc[i,stoch] <= 100:
            df.iloc[i,stoch] = -2
        elif 0 <= df.iloc[i,stoch] <= 20:
            df.iloc[i,stoch] = 2
        elif 20 < df.iloc[i,stoch] <= 30:
            df.iloc[i,stoch] = 1
        elif 70 <= df.iloc[i,stoch] < 80:
            df.iloc[i,stoch] = -1
        else:
            df.iloc[i,stoch] = 0
    
    #Mapping TSI values
    tsi = df.columns.get_loc('tsi')
    for i in range(length):
        if df.iloc[i,tsi] < -25:
            df.iloc[i,tsi] = -2
        elif df.iloc[i,tsi] <= -10:
            df.iloc[i,tsi] = -1
        elif df.iloc[i,tsi] <= 10:
            df.iloc[i,tsi] = 0
        elif df.iloc[i,tsi] < 25:
            df.iloc[i,tsi] = 1
        else:
            df.iloc[i,tsi] = 2
    
    #Mapping MACD values
    macd = df.columns.get_loc('MACD_12_26')
    for i in range(length):
        if 0 < df.iloc[i,macd]:
            df.iloc[i,macd] = 1
        elif df.iloc[i,macd] < 0:
            df.iloc[i,macd] = -1
        else:
            df.iloc[i,macd] = 0
            
    #Mapping TRIX values
    trix = df.columns.get_loc('trix_15')
    for i in range(length):
        if 0 < df.iloc[i,trix]:
            df.iloc[i,trix] = 1
        elif df.iloc[i,trix] < 0:
            df.iloc[i,trix] = -1
        else:
            df.iloc[i,trix] = 0
            
    #Mapping Aroon values
    values = []
    aroon_up = df.columns.get_loc('aroon_up25')
    aroon_down = df.columns.get_loc('aroon_down25')
    for i in range(length):
        if df.iloc[i,aroon_up] >= 70 and df.iloc[i,aroon_down] <= 30:
            values.append(2.0)
        elif df.iloc[i,aroon_down] >= 70 and df.iloc[i,aroon_up] <= 30:
            values.append(-2.0)
        elif df.iloc[i,aroon_up] > df.iloc[i,aroon_down]:
            values.append(1.0)
        elif df.iloc[i,aroon_down] > df.iloc[i,aroon_up]:
            values.append(-1.0)
        else:
            values.append(0.0)
    df['aroon'] = values
    
    #Mapping Chaikin values
    cmf = df.columns.get_loc('cmf')
    for i in range(length):
        if 0.05 <= df.iloc[i,cmf] < 0.1:
            df.iloc[i,cmf] = 1
        elif -0.1 < df.iloc[i,cmf] <= -0.05:
            df.iloc[i,cmf] = -1
        elif df.iloc[i,cmf] >= 0.1:
            df.iloc[i,cmf] = 2
        elif df.iloc[i,cmf] <= -0.1:
            df.iloc[i,cmf] = -2
        else:
            df.iloc[i,cmf] = 0
    
    #Mapping rsi values
    rsi = df.columns.get_loc('rsi')
    for i in range(length):
        if 70 <= df.iloc[i,rsi] < 100:
            df.iloc[i,rsi] = -2
        elif 0 <= df.iloc[i,rsi] <= 30:
            df.iloc[i,cmf] = 2
        elif 30 < df.iloc[i,rsi] >= 40:
            df.iloc[i,rsi] = 1
        elif 60 <= df.iloc[i,rsi] < 70:
            df.iloc[i,rsi] = -1
        else:
            df.iloc[i,rsi] = 0
            
    del df['aroon_up25']
    del df['aroon_down25']

    return df

In [12]:
test = daily_df('AAPL')
test2 = run_indicators(test)

,open,high,low,close,volume,wr,mfi_14,stoch_d,tsi,MACD_12_26,trix_15,aroon_up25,aroon_down25,cmf,bbi,kci,rsi
2019-10-25,243.16,246.72,242.89,246.58,1.58277e+07,-0.625279,78.839371,97.701567,22.802090,6.642883,0.368968,100.0,8.0,0.113703,0.0,0.0,76.654622
2019-10-24,244.51,244.8,241.81,243.58,1.73188e+07,-5.95994,79.677086,95.151713,21.727455,6.315888,0.354970,100.0,12.0,0.075124,0.0,0.0,74.179051
2019-10-23,242.1,243.24,241.22,243.18,1.89572e+07,-0.310078,80.580307,96.519644,20.817717,6.111974,0.342188,100.0,16.0,0.068915,0.0,0.0,73.835540
2019-10-22,241.16,242.2,239.62,239.96,2.05734e+07,-8.27484,73.420132,94.130249,19.847619,5.803504,0.329285,96.0,20.0,0.067274,0.0,0.0,70.946320
2019-10-21,237.52,240.99,237.32,240.51,2.18118e+07,-1.85615,65.350606,92.758095,19.076799,5.653055,0.317651,100.0,24.0,0.046567,0.0,0.0,72.211041


In [10]:
test3 = map_values(test2)
test3.head(5)

AttributeError: 'Series' object has no attribute 'add_format'

In [ ]:
def run_all():
    df = daily_df('AAPL')
    df = run_indicators(test)

In [13]:
layout = [      
          [sg.Text('Please enter your Name, Address, Phone')],      
          [sg.Text('Name', size=(15, 1)), sg.InputText('name')],      
          [sg.Text('Address', size=(15, 1)), sg.InputText('address')],      
          [sg.Text('Phone', size=(15, 1)), sg.InputText('phone')],      
          [sg.Submit(), sg.Cancel()]      
         ]

window = sg.Window('Simple data entry window').Layout(layout)         
button, values = window.Read()

print(button, values[0], values[1], values[2])

In [13]:
show(test3)

Using existing QApplication instance


In [4]:
widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)


Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

In [6]:
import sys
print(sys.path)

['/private/var/folders/9w/qrsycnrj52g4lm658t6wng3m0000gn/T/f77e0f04-d0fb-4c7b-89b1-2b7b904f1bdb', '/Users/rohitgarudadri/anaconda3/lib/python37.zip', '/Users/rohitgarudadri/anaconda3/lib/python3.7', '/Users/rohitgarudadri/anaconda3/lib/python3.7/lib-dynload', '', '/Users/rohitgarudadri/anaconda3/lib/python3.7/site-packages', '/Users/rohitgarudadri/anaconda3/lib/python3.7/site-packages/aeosa', '/Users/rohitgarudadri/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/Users/rohitgarudadri/.ipython']
